In [1]:
// 怎样才会释放内存
// 高级主题
// JsonParams case 实战

In [1]:
# include <mutex>
# include <iostream>
# include <memory>
#include <cstdint>

In [12]:
uintptr_t debug_p;
int value;
int a;


In [2]:
a = 3;
int *p = &a;
// 不是动态内存的时候好像也OK？
int *p2 = new int(3);  
// 必须是动态内存
std::cout << p << std::endl;
std::cout << p2 << std::endl;

In [4]:
// 普通指针初始化，new初始化，reset初始化

std::shared_ptr<int> sp2(p);
std::shared_ptr<int> sp3(p2);

std::cout << p2 << std::endl;
// 初始化智能指针可以用已经存在的基本指针，但是一旦智能指针释放，基本指针也将无法取到值，是风险所在
// shared_ptr绑定普通指针，则承担管理责任，一旦使用，内置指针则废弃。

In [ ]:
sp2 = new int(3); 
sp2.reset(new int(3)); //  pointer being freed was not allocated. 已经赋值的指针reset失败

In [2]:
std::shared_ptr<int> sp4;
sp4.reset(new int(5)); // 没有赋值过的指针可以用reset赋值
std::cout << sp4 << std::endl;
// sp4.reset(); // 也可以释放
sp4.reset(new int(8)); // 也可以重新指向
std::cout << sp4 << std::endl;

In [5]:
// make_shared初始化
std::shared_ptr<int> sp3 = std::make_shared<int> (42);

In [ ]:
// ============================================ 怎样才会释放内存

In [2]:
// ------------------------------------ Case 1
int a = 1;
auto sp4 = std::make_shared<int>(a);
std::cout << sp4 << std::endl;
int *q = &a; 
std::cout << q << std::endl;
sp4.reset(); // 没有释放？是因为这种情况下 *q  指向的是栈上的内存，并未指向堆；a相当于只是作为一个值传入

0x7fd31d6c2448
0x111a3a410


In [10]:
debug_p = 0x7fd31d6c2448;
value = *reinterpret_cast<int *>(debug_p); 
value
// 动态内存本身其实已经释放

4

In [2]:
// ------------------------------------ Case 1.5

void process(std::shared_ptr<int> ptr){
    std::cout << ptr << std::endl;
    std::cout << *ptr << std::endl;
} // 进函数时才初始化，计数 = 1，则出函数后释放空间


In [3]:
int *x(new int(5));
std::cout << x << std::endl;
// process(x); 无法隐式转换
process(std::shared_ptr<int>(x)); // 强制转换得到的智能指针也被绑定到了相应的位置（和x相同的一块动态内存）上

std::cout << x << std::endl;   // 指向位置没发生变化
std::cout << *x << std::endl; // 但相应位置的内存空间已经释放

0x7fb12a221be0
0x7fb12a221be0
5


In [30]:
// ------------------------------------ Case 2

auto sp5 = std::make_shared<int>(3);
auto sp6(sp5);
// int *a_q = *sp6; 错误
int *a_q = sp6.get();

In [32]:
sp6.reset();
std::cout << *a_q << std::endl;
sp5.reset();
*a_q;
// 似乎在这种纯动态内存情况下依然未释放

In [4]:
std::shared_ptr<int> p(new int(42));
int *q = p.get();
std::cout << q << std::endl;
{
    std::shared_ptr<int>(q);
    std::cout << q << std::endl;
}
std::cout << p << endl;
// q 开始时和 p 保存了相同地址，在强行转换操作中失去地址；p的地址仍在

0x7fd92f1b56f0
0x0


42

In [8]:
// ------------------------------------ Case 3: 验证最简单情形下的动态内存释放

std::shared_ptr<int> a_p(new int(42));
std::cout << a_p << std::endl;

0x7fd92f1e2f60


In [10]:
debug_p = 0x7fd92f1e2f60;
value = *reinterpret_cast<int *>(debug_p);
value

In [12]:
a_p.reset();
*reinterpret_cast<int *>(debug_p); // 动态内存确实释放了

In [ ]:
// ============================================ 高级主题

In [14]:
// 多路shared_ptr绑定同一块内存


0x0


In [ ]:
// shared_ptr强制转换导致原指针无效 体现风险的例子

In [27]:
// 其他类型资源的绑定

In [ ]:
// ========================================== JsonParams case 实战

// 把动态指针拷贝给动态指针： clone返回物作为动态指针是ok的

std::shared_ptr<int> p(new int(3));
std::shared_ptr<int> qq = p;
std::cout << p << std::endl;
std::cout << qq << std::endl;

// 也符合原则：动态指针甫一创建，之后这块空间的管理就交给动态指针们，其他人只能从它处拷贝
// 利用同一个managed_object & manager_object，保留计数器的原理来实现没有动态指针后就释放这块空间

In [9]:
int* foo = new int(3);
std::shared_ptr<int> ref_a(foo);
std::shared_ptr<int> ref_b(foo); // 你死了
delete foo;
ref_a.reset(new int(3)) // 如果它的空间已经被释放过了，就会core

In [14]:
std::cout << ref_a<< std::endl;
std::cout << ref_b<< std::endl;
std::cout << foo << std::endl;

0x7fb12a9479a0
0x7fb12a9479a0
0x7fb12a9479a0
